<a href="https://colab.research.google.com/github/YolandaMDavis/cs109a-fall2020/blob/covid-research/Boston_Covid_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# How to use Geohash - https://bigfastblog.com/geohash-intro

!pip install pygeohash

  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6162 sha256=f0e9119dc9712bd229adba8422b498ecbefd30c2aec3de1dd4c7a68e410789d9
  Stored in directory: /root/.cache/pip/wheels/3f/5f/14/989d83a271207dda28232746d63e737a2dbd88ea7f7a9db807
Successfully built pygeohash


In [3]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pygeohash as pgh
DATA_ROOT = '/content/drive/My Drive/Final-Project-for-CS109a/covid_analysis/'

In [20]:
boston_covid_neighborhood_df = pd.read_csv(DATA_ROOT + 'boston_neighborhood_covid.csv')
boston_covid_neighborhood_df["total positive"] = boston_covid_neighborhood_df["total tested"] * boston_covid_neighborhood_df["cumulative positive"]
boston_covid_neighborhood_df = boston_covid_neighborhood_df.rename(columns={"neigborhood":"neighborhood"})
boston_covid_neighborhood_df.head()


,neighborhood,total tested,cumulative positive,total positive
0,East Boston,19590,0.13600,2664.2400
1,Mattapan,8652,0.11200,969.0240
2,Hyde Park,12726,0.11100,1412.5860
3,Dorchester,48850,0.10751,5251.8635
4,Roslindale,12005,0.08200,984.4100


In [21]:
boston_districts_df = pd.read_excel(DATA_ROOT + 'Boston-Neighborhoods-to-Districts.xlsx')
boston_districts_df

,Unnamed: 0,neighborhood,district
0,0,Allston,D14
1,1,Back Bay,D4
2,2,Beacon Hill,A1
3,3,Brighton,D14
4,4,Charlestown,A15
5,5,Dorchester,C11
6,6,Downtown,A1
7,7,East Boston,A7
8,8,Fenway,D4
9,9,Harbor Islands,NaN


In [22]:
boston_covid_districts = pd.merge(boston_covid_neighborhood_df,boston_districts_df, on="neighborhood").drop(columns=["Unnamed: 0"])
boston_covid_districts

,neighborhood,total tested,cumulative positive,total positive,district
0,East Boston,19590,0.136,2664.240,A7
1,Roslindale,12005,0.082,984.410,E5
2,Roxbury,23736,0.062,1471.632,B2
3,West Roxbury,9356,0.061,570.716,E5
4,South End,20060,0.057,1143.420,D4
5,Jamaica Plain,19720,0.049,966.280,E13
6,South Boston,16450,0.047,773.150,C6
7,Allston,12753,0.042,535.626,D14
8,Brighton,18697,0.042,785.274,D14
9,Charlestown,7060,0.039,275.340,A15


In [23]:
boston_covid_districts = boston_covid_districts.groupby("district").sum()[["total positive","total tested","cumulative positive"]]
boston_covid_districts

,total positive,total tested,cumulative positive
district,,,
A1,476.523,17649,0.108
A15,275.340,7060,0.039
A7,2664.240,19590,0.136
B2,1471.632,23736,0.062
C6,773.150,16450,0.047
D14,1320.900,31450,0.084
D4,1971.395,70570,0.098
E13,966.280,19720,0.049
E5,1555.126,21361,0.143


In [38]:
boston_crime_df = pd.read_csv(DATA_ROOT + 'boston_crime_2020_latest.csv')
boston_crime_df.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,TESTTEST2,423,NaN,ASSAULT - AGGRAVATED,External,,0,2019-10-16 00:00:00,2019,10,Wednesday,0,NaN,RIVERVIEW DR,NaN,NaN,"(0.00000000, 0.00000000)"
1,S97333701,3301,NaN,VERBAL DISPUTE,C6,915,0,2020-07-18 14:34:00,2020,7,Saturday,14,NaN,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)"
2,S47513131,2647,NaN,THREATS TO DO BODILY HARM,E18,530,0,2020-06-24 10:15:00,2020,6,Wednesday,10,NaN,READVILLE ST,42.239491,-71.135954,"(42.23949100, -71.13595400)"
3,I92102201,3301,NaN,VERBAL DISPUTE,E13,583,0,2019-12-20 03:08:00,2019,12,Friday,3,NaN,DAY ST,42.325122,-71.107779,"(42.32512200, -71.10777900)"
4,I92097173,3115,NaN,INVESTIGATE PERSON,C11,355,0,2019-10-23 00:00:00,2019,10,Wednesday,0,NaN,GIBSON ST,42.297555,-71.059709,"(42.29755500, -71.05970900)"


In [30]:
crime_violence_ind = pd.read_csv(DATA_ROOT + 'boston_crime_violence_ind.csv')
crime_violence_ind["OFFENSE_TYPE"].unique()

array(['Assault and/or Battery', 'Other', 'Animal Crime',
       'MV Accident/Property Damage', 'Theft/Robbery/B&E',
       'Child Abuse/Endangerment', 'Harassment',
       'Murder/Manslaughter/Homocide', 'Drugs/Alcohol', 'Weapon',
       'Sex Offense', 'Forgery/Fraud', 'Kidnapping', 'Missing Person',
       'Prisoner Offense', 'Prostitution', 'Val', 'Violation of Order',
       'Warrant'], dtype=object)

In [39]:
# Select 2020 crimes
boston_crime_df = boston_crime_df.drop(columns=["OFFENSE_CODE_GROUP","REPORTING_AREA","UCR_PART"])

#drop district data
boston_crime_df.dropna(subset=['DISTRICT'], inplace=True)

print(boston_crime_df["DISTRICT"].unique())

# Merge with Covid Data
boston_crime_df = pd.merge(boston_crime_df,boston_covid_districts, left_on="DISTRICT", right_on="district", how="outer")

# Merge with new Violent Crime Indicator
boston_crime_df = pd.merge(boston_crime_df,crime_violence_ind, left_on="OFFENSE_DESCRIPTION", right_on="OFFENSE_DESCRIPTION")

# Cleanup shooting data using 0 (no shooting), 1 (shooting), 2 (unknown)
boston_crime_df["SHOOTING"].fillna(value=2, inplace=True)
boston_crime_df["SHOOTING"].replace(to_replace='0',value=0.0, inplace=True)
boston_crime_df["SHOOTING"].replace(to_replace='Y',value=1.0, inplace=True)

# Set any shooting indicator as violent crime
filter = boston_crime_df["SHOOTING"] == 1
boston_crime_df.loc[filter,"VIOLENT_CRIME"] = 1



['External' 'C6' 'E18' 'E13' 'C11' 'D14' 'B2' 'A15' 'E5' 'A1' 'B3' 'A7'
 'D4']


In [40]:
boston_crime_df = boston_crime_df.rename(columns={"total positive": "COVID_POSITIVE_COUNT", "total tested": "COVID_TESTED_COUNT", "cumulative positive": "COVID_POSITIVE_RATE","Lat":"LATITUDE","Long":"LONGITUDE","Location":"LOCATION"})
boston_crime_df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE
0,TESTTEST2,423,ASSAULT - AGGRAVATED,External,0.0,2019-10-16 00:00:00,2019,10,Wednesday,0,RIVERVIEW DR,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery
1,202062971,423,ASSAULT - AGGRAVATED,External,0.0,2020-08-30 17:35:00,2020,8,Sunday,17,RUGGLES ST,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery
2,202061413,423,ASSAULT - AGGRAVATED,External,0.0,2020-07-27 11:10:00,2020,7,Monday,11,WASHINGTON STREET,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery
3,202060210,423,ASSAULT - AGGRAVATED,External,0.0,2020-08-20 21:52:00,2020,8,Thursday,21,PROVIDENCE HWY,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery
4,I192078179,423,ASSAULT - AGGRAVATED,C6,2.0,2019-09-27 17:59:00,2019,9,Friday,17,MOHAWK ST,42.328937,-71.055717,"(42.32893710, -71.05571742)",773.15,16450.0,0.047,1,Assault and/or Battery


In [41]:
boston_crime_df["DISTRICT"].unique()

array(['External', 'C6', 'E18', 'E13', 'C11', 'D14', 'B2', 'A15', 'E5',
       'A1', 'B3', 'A7', 'D4'], dtype=object)

In [42]:
# Determine Covid Period indicator based on Boston emergency order date March 17th, 2020
import datetime
lockdown_date = datetime.datetime(2020, 3, 17)
crime_dates = pd.to_datetime(boston_crime_df["OCCURRED_ON_DATE"])
covid_ind = crime_dates >= lockdown_date
boston_crime_df["COVID_PERIOD"] = covid_ind.astype(int)
boston_crime_df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE,COVID_PERIOD
0,TESTTEST2,423,ASSAULT - AGGRAVATED,External,0.0,2019-10-16 00:00:00,2019,10,Wednesday,0,RIVERVIEW DR,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,0
1,202062971,423,ASSAULT - AGGRAVATED,External,0.0,2020-08-30 17:35:00,2020,8,Sunday,17,RUGGLES ST,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,1
2,202061413,423,ASSAULT - AGGRAVATED,External,0.0,2020-07-27 11:10:00,2020,7,Monday,11,WASHINGTON STREET,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,1
3,202060210,423,ASSAULT - AGGRAVATED,External,0.0,2020-08-20 21:52:00,2020,8,Thursday,21,PROVIDENCE HWY,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,1
4,I192078179,423,ASSAULT - AGGRAVATED,C6,2.0,2019-09-27 17:59:00,2019,9,Friday,17,MOHAWK ST,42.328937,-71.055717,"(42.32893710, -71.05571742)",773.15,16450.0,0.047,1,Assault and/or Battery,0


In [43]:
boston_crime_df.loc[boston_crime_df["COVID_PERIOD"] == 0, ["COVID_POSITIVE_COUNT","COVID_TESTED_COUNT", "COVID_POSITIVE_RATE"]] = np.NaN, np.NaN, np.NaN
boston_crime_df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE,COVID_PERIOD
0,TESTTEST2,423,ASSAULT - AGGRAVATED,External,0.0,2019-10-16 00:00:00,2019,10,Wednesday,0,RIVERVIEW DR,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,0
1,202062971,423,ASSAULT - AGGRAVATED,External,0.0,2020-08-30 17:35:00,2020,8,Sunday,17,RUGGLES ST,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,1
2,202061413,423,ASSAULT - AGGRAVATED,External,0.0,2020-07-27 11:10:00,2020,7,Monday,11,WASHINGTON STREET,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,1
3,202060210,423,ASSAULT - AGGRAVATED,External,0.0,2020-08-20 21:52:00,2020,8,Thursday,21,PROVIDENCE HWY,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,1
4,I192078179,423,ASSAULT - AGGRAVATED,C6,2.0,2019-09-27 17:59:00,2019,9,Friday,17,MOHAWK ST,42.328937,-71.055717,"(42.32893710, -71.05571742)",NaN,NaN,NaN,1,Assault and/or Battery,0


In [44]:
boston_crime_df["GEOHASH"] = [pgh.encode(row["LATITUDE"],row["LONGITUDE"]) for idx, row in boston_crime_df.iterrows()]
boston_crime_df.sort_values(by=["GEOHASH"])

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE,COVID_PERIOD,GEOHASH
0,TESTTEST2,423,ASSAULT - AGGRAVATED,External,0.0,2019-10-16 00:00:00,2019,10,Wednesday,0,RIVERVIEW DR,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,1,Assault and/or Battery,0,000000000000
388693,I182022517,3821,M/V ACCIDENT - INVOLVING PEDESTRIAN - NO INJURY,A7,2.0,2018-03-26 13:56:00,2018,3,Monday,13,MERIDIAN ST,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,0,MV Accident/Property Damage,0,000000000000
388692,I182104165,3821,M/V ACCIDENT - INVOLVING PEDESTRIAN - NO INJURY,A7,2.0,2018-12-27 12:46:00,2018,12,Thursday,12,BENNINGTON ST,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,0,MV Accident/Property Damage,0,000000000000
388666,I172041456,3821,M/V ACCIDENT - INVOLVING PEDESTRIAN - NO INJURY,B3,2.0,2017-05-25 19:59:00,2017,5,Thursday,19,BLUE HILL AVE,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,0,MV Accident/Property Damage,0,000000000000
388664,I172074396,3821,M/V ACCIDENT - INVOLVING PEDESTRIAN - NO INJURY,B3,2.0,2017-09-08 08:37:00,2017,9,Friday,8,GLENWAY ST,NaN,NaN,"(0.00000000, 0.00000000)",NaN,NaN,NaN,0,MV Accident/Property Damage,0,000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83541,I162104603,3006,SICK/INJURED/MEDICAL - PERSON,A7,2.0,2016-12-23 12:00:00,2016,12,Friday,12,WALDEMAR AVE,42.391463,-71.001342,"(42.39146297, -71.00134187)",NaN,NaN,NaN,0,Other,0,drt90m4khv6q
45319,I182087526,3115,INVESTIGATE PERSON,A7,2.0,2018-10-30 07:28:00,2018,10,Tuesday,7,WALDEMAR AVE,42.391463,-71.001342,"(42.39146297, -71.00134187)",NaN,NaN,NaN,0,Other,0,drt90m4khv6q
46295,202055831,3115,INVESTIGATE PERSON,A7,0.0,2020-08-05 15:53:00,2020,8,Wednesday,15,BENNINGTON ST,42.391932,-70.995361,"(42.39193200, -70.99536100)",2664.24,19590.0,0.136,0,Other,1,drt90mnybhvu
309821,I162043831,3802,M/V ACCIDENT - PROPERTY DAMAGE,A7,2.0,2016-06-03 22:10:00,2016,6,Friday,22,BENNINGTON ST,42.391933,-70.995362,"(42.39193303, -70.99536185)",NaN,NaN,NaN,0,MV Accident/Property Damage,0,drt90mnybjj6


In [45]:
boston_crime_df.shape

(523627, 21)

In [46]:
# Write out file (keep commented out unless updating)
boston_crime_df.to_csv(DATA_ROOT + 'boston_crime_covid_full.csv')